# Simple linear regression

In [ ]:
import warnings
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [ ]:
from statsmodels.compat import lzip
import statsmodels.api as sm
import statsmodels.stats.api as sms
warnings.filterwarnings("ignore")

In [ ]:
# Datase
diabetes = pd.read_csv(os.path.join('data', 'diabetes.csv'))

In [ ]:
diabetes.drop(columns=['y'], inplace=True)

# Objective

* To explain the relationship between glucose and body mass index (BMI).

## Questions

* Is there a relationship between glucose and BMI?
* The relationship is linear?
* Body mass index (BMI) has an effect on glucose?
* If the effect exists, is positive or negative?

## Methodology

* Simple linear regression

y = dependent variable (response variable) = glucose

x = independent variable (predictor variable) = BMI

$\beta_0$ is the intercept term, $\beta_1$ is the slope term.

* Hypothesis testing

$H_0$ = there is no relationship between glucose and BMI

$H_1$ = there is a relationship between glucose and BMI

### a) Explore the data

In [ ]:
diabetes.describe()

In [ ]:
corr = diabetes.corr()
corr

In [ ]:
plt.subplots(figsize=(8,8))
sns.heatmap(corr,annot=True)
plt.show()

### b) Linear regression with one variable

In [ ]:
# series into numpy array
bmi = diabetes.bmi.to_numpy()
glucose = diabetes.glu.to_numpy()

In [ ]:
# define the input variable (X) & output variable (Y)
X = bmi
Y = glucose

In [ ]:
# Plot all the data
sns.scatterplot(x=X, y=Y)
plt.show()

#### Option 1. Using Scikit learn

In [ ]:
# Linear Regression Model
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
rls = linear_model.LinearRegression()
modelo = rls.fit(np.reshape(X_train,(-1,1)),Y_train)
Y_pred = rls.predict(np.reshape(X_test,(-1,1)))

In [ ]:
intercept = rls.intercept_
slope = rls.coef_

intercept, slope

### To see step by step (results for R-squared of test data)

In [ ]:
df_test = pd.DataFrame({'x': X_test, 'y': Y_test, 'y_pred': Y_pred})

In [ ]:
# Plot the data
sns.scatterplot(x=X_test, y=Y_test)
plt.show()

In [ ]:
df_test['residual_squares'] = (df_test['y'] - df_test['y_pred'])**2
df_test['sum_of_squares'] = (df_test['y'] - df_test['y'].mean())**2

In [ ]:
df_test

In [ ]:
rss_test = df_test['residual_squares'].sum()
tss_test = df_test['sum_of_squares'].sum()

In [ ]:
r2_test = 1 - (rss_test/tss_test)
r2_test

### To see step by step (results for R.squared of training data)

In [ ]:
df_train = pd.DataFrame({'x': X_train, 'y': Y_train})

In [ ]:
# Plot the data
sns.scatterplot(x=X_train, y=Y_train)
plt.show()

In [ ]:
df_train['y_pred'] = slope*X_train + intercept

In [ ]:
df_train['residual_squares'] = (df_train['y'] - df_train['y_pred'])**2
df_train['sum_of_squares'] = (df_train['y'] - df_train['y'].mean())**2

In [ ]:
df_train

In [ ]:
rss_train = df_train['residual_squares'].sum()
tss_train = df_train['sum_of_squares'].sum()

In [ ]:
r2_train = 1 - (rss_train/tss_train)
r2_train

In [ ]:
# Parameters for hypothesis test B1
# Parámetros para prueba de hipótesis B1
error = Y_test - Y_pred
ds_error = error.std()
ds_X = X_test.std()
error_st = ds_error/np.sqrt(102)
t1 = rls.coef_/(error_st/ds_X)
print(t1)

In [ ]:
# Parameters for hypothesis test B0
# Parámetros para prueba de hipótesis B0
media_X = X_test.mean()
media_XC = pow(media_X,2)
var_X = X_test.var()
to = rls.intercept_/(error_st*np.sqrt(1+(media_XC/var_X)))
print(to)

In [ ]:
#%% Linear regression graph
plt.scatter(X_test,Y_test)
plt.plot(X_test,Y_pred, color='r',linewidth=3)
plt.title(' Linear Regression ')
plt.xlabel('Body Mass Index (BMI)')
plt.ylabel('Glucose (mg) in blood')
plt.show()

In [ ]:
# Ajuste de la linea de regresión
plt.figure()
sns.regplot(Y_test,Y_pred, data=diabetes, marker='+')
plt.xlabel('Actual Values')
plt.ylabel('Predicted  Values')
plt.title('Actual Values VS Predicted Value')
plt.show()

#### Option 2. Statsmodel

In [ ]:
# Modelo de regresión lineal (statsmodel)
X_2=sm.add_constant(X_train,prepend=True)
rls_2=sm.OLS(Y_train,X_2)
modelo_2=rls_2.fit()
print(modelo_2.summary())
Y_pred_2=modelo_2.predict()
error_2=modelo_2.resid

In [ ]:
# Visualize homoscedasticity
# Visualizar homocedasticidad 
plt.figure()
sns.regplot(Y_pred_2,error_2, data=diabetes, marker='*')
plt.xlabel('Fitted Values', size=20)
plt.ylabel('Residuals', size=20)
plt.title('Fitted Values VS Residuals', size=20)
plt.show()

In [ ]:
# Forma Estadística de Homocedasticidad
# Breusch-Pagan
# H0: Homocedasticidad (p>0.05)
# H1: No homocedasticidad (p<0.05)

# H0: Homoscedasticity (p>0.05)
# H1: No homoscedasticity (p<0.05)
names=['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(modelo_2.resid, X_2)
lzip(names, test)

In [ ]:
# Plot of normality of residuals
# Forma gráfica de la  normalidad de los residuos
plt.figure()
plt.hist(modelo_2.resid)
plt.show()

In [ ]:
# QQ plot
plt.figure()
ax=sm.qqplot(modelo_2.resid)
plt.show()

In [ ]:
# QQ plot
plt.figure()

ax=sm.qqplot(modelo_2.resid, line='45',scale=10)
plt.show()

In [ ]:
# Forma estadística de la normalidda (Shapiro-Wilk)
#Ho: Normalidad (p>0.05)
#H1: No normalidad (p<0.05)

#Ho: Normality (p>0.05)
#H1: No normality (p<0.05)
names=[' Statistic', 'p-value']
test=stats.shapiro(modelo_2.resid)
lzip(names,test)

# Multiple linear regression

In [ ]:
# Convert series to numpy array
diabetes_array = diabetes.to_numpy()

In [ ]:
# Dimensions of the array
diabetes_array.shape

In [ ]:
# Glucose as dependent variable and the rest are part of the model of variables
X_1 = diabetes.drop(labels='glu', axis=1)
Y_1 = diabetes['glu']

##### Statsmodel

In [ ]:
lm2 = sm.OLS.from_formula("glu ~ age+sex+bmi+bp+tc+ldl+hdl+tch+ltg", data=diabetes)

In [ ]:
trained_lm2 = lm2.fit()

In [ ]:
predictions = trained_lm2.predict(X_1)

In [ ]:
summary_model = trained_lm2.summary()
print(summary_model)

In [ ]:
# Forma Estadística de Homocedasticidad
#Breusch-Pagan
#H0: Homocedasticidad (p>0.05)
#H1: No homocedasticidad (p<0.05)

#H0: Homoscedasticity (p>0.05)
#H1: No homoscedasticity (p<0.05)

names = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(trained_lm2.resid, trained_lm2.model.exog)

lzip(names, test)

#### Model considering only the significant variables from previous

In [ ]:
lm3 = sm.OLS.from_formula("glu ~ age+bmi+bp", data=diabetes)

In [ ]:
trained_lm3 = lm3.fit()

In [ ]:
predictions = trained_lm3.predict(X_1)

In [ ]:
summary_model_0 = trained_lm3.summary()
print(summary_model_0)